## Learning Convolutional Kernels with MIP

In [1]:
import numpy as np
from docplex.mp.model import Model

def kernel_mip(data, tau):
    model = Model(name = 'Sparse Autoregression')
    T = data.shape[0]
    w = [model.continuous_var(lb = 0, name = f'w_{k}') for k in range(T - 1)]
    beta = [model.binary_var(name = f'beta_{k}') for k in range(T - 1)]
    error = [data[t] - model.sum(w[k] * data[t - k - 1] for k in range(T - 1)) for t in range(T)]
    model.minimize(model.sum(r ** 2 for r in error))
    model.add_constraint(model.sum(beta[k] for k in range(T - 1)) <= tau)
    for k in range(T - 1):
        model.add_constraint(w[k] <= beta[k])
    solution = model.solve()
    if solution:
        w_coef = np.array(solution.get_values(w))
        error = 0
        for t in range(T):
            a = data[t]
            for k in range(T - 1):
                a -= w_coef[k] * data[t - k - 1]
            error += a ** 2
        print('Objective function: {}'.format(error))
        ind = np.where(w_coef > 0)[0].tolist()
        print('Support set: ', ind)
        print('Coefficients w: ', w_coef[ind])
        print('Cardinality of support set: ', len(ind))
        return w_coef, ind
    else:
        print('No solution found.')
        return None

## Learning Convolutional Kernels with NNSP

In [2]:
import numpy as np
from scipy.optimize import nnls

def circ_mat(vec):
    n = vec.shape[0]
    mat = np.zeros((n, n))
    mat[:, 0] = vec
    for i in range(1, n):
        mat[:, i] = np.append(vec[-i :], vec[: -i])
    return mat

def SP(A, x, tau, stop = np.infty, nonnegative = True, epsilon = 1e-2):
    m, n = A.shape
    r = x
    w = np.zeros(n)
    S = np.array([])
    i = 0
    while np.linalg.norm(r, 2) > epsilon and i < stop:
        Ar = A.T @ r
        S0 = np.argsort(abs(Ar))[- tau :]
        S = np.append(S[:], S0[:]).astype(int)
        if nonnegative == True:
            w[S], _ = nnls(A[:, S], x)
        elif nonnegative == False:
            w[S] = np.linalg.pinv(A[:, S]) @ x
        S = np.argsort(abs(w))[- tau :]
        w = np.zeros(n)
        if nonnegative == True:
            w[S], _ = nnls(A[:, S], x)
        elif nonnegative == False:
            w[S] = np.linalg.pinv(A[:, S]) @ x
        r = x - A[:, S] @ w[S]
        i += 1
    return w, S, r

## Two-Week Trip Time Series of Chicago

### Optimization with MIP

In [3]:
import numpy as np
import time

tensor = np.load('../Chicago-data/Chicago_rideshare_mob_tensor_24.npz')['arr_0'][:, :, : 14 * 24]
data = np.sum(np.sum(tensor, axis = 0), axis = 0)
tau = 3

start = time.time()
w, ind = kernel_mip(data, tau)
end = time.time()
print('Running time (s):', end - start)

Objective function: 76747968.5653466
Support set:  [0, 167, 334]
Coefficients w:  [0.33677239 0.33329912 0.33677239]
Cardinality of support set:  3
Running time (s): 30.78903079032898


### Optimization with NNSP

In [4]:
import numpy as np
import time

tensor = np.load('../Chicago-data/Chicago_rideshare_mob_tensor_24.npz')['arr_0'][:, :, : 14 * 24]
data = np.sum(np.sum(tensor, axis = 0), axis = 0)
tau = 3

start = time.time()
w, S, r = SP(circ_mat(data)[:, 1 :], data, tau, 5, True)
end = time.time()
print('Indices of non-zero coefficients (support set):', S)
print('Non-zero entries of sparse temporal kernel:', w[S])
print('Loss function:', np.linalg.norm(r, 2) ** 2)
print('Running time (s):', end - start)

Indices of non-zero coefficients (support set): [167 334   0]
Non-zero entries of sparse temporal kernel: [0.33329912 0.33677239 0.33677239]
Loss function: 76747968.56534657
Running time (s): 0.001653909683227539


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>